In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
import torchvision.transforms as T
from torchvision import datasets
#from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torchvision.io import read_image
#from IPython.display import Image
#from pandas.core.common import flatten
import torchvision.transforms.functional as fn
from torch import optim
import torch.nn.functional as F
#from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#import albumentations as A
#from albumentations.pytorch import ToTensorV2
import datetime
import cv2
import glob
import numpy as np
import imageio
import os
import copy
from os import walk
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.models as models

In [3]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device

In [3]:
df = pd.read_csv('captionsfolder/results.csv', sep='|')
#df = df.iloc[:56]
df

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ov...
158911,998845445.jpg,1,"A young man hanging over the side of a boat ,..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue an...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with ..."


In [9]:
df.dropna(inplace=True)

In [30]:
def image_data_preprocessor(image):
    img_data = Image.open("images/flickr30k_images/" + image)
    image_transformer = T.Compose([
                        T.Resize((256, 256)),
                        #T.RandomCrop(299, 299),
                        T.ToTensor(),
                        T.Normalize(
                            mean = [0.5, 0.5, 0.5],
                            std = [0.5, 0.5, 0.5]),
                        T.Grayscale()
                   
                        
    ])
    transformed_image_data = image_transformer(img_data)
    return transformed_image_data

In [11]:
# Tokenize the captions
vocab = set()
for caption in df['comment']:
    #print(caption)
    vocab.update(caption.split())

# Create a vocabulary of the most common words
vocab = list(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

In [12]:
#vocab
word_to_idx
print(vocab[0])

Dental


In [13]:
# Map each word in the captions to an integer index
captions = []
for caption in df['comment']:
    tokens = caption.split()
    indices = [word_to_idx[token] for token in tokens]
    captions.append(indices)
 

lengths_of_captions = []
for caption_data in captions:
    lengths_of_captions.append(len(caption_data))
lengths_of_captions


[17,
 11,
 11,
 11,
 7,
 12,
 12,
 10,
 9,
 7,
 18,
 13,
 10,
 8,
 8,
 17,
 15,
 13,
 10,
 8,
 20,
 10,
 11,
 9,
 7,
 17,
 15,
 9,
 9,
 5,
 16,
 13,
 11,
 9,
 8,
 15,
 14,
 14,
 11,
 8,
 17,
 13,
 14,
 12,
 7,
 15,
 12,
 11,
 10,
 7,
 17,
 11,
 11,
 10,
 10,
 23,
 16,
 16,
 14,
 13,
 19,
 15,
 13,
 9,
 9,
 18,
 17,
 16,
 13,
 15,
 14,
 11,
 12,
 8,
 4,
 20,
 17,
 12,
 12,
 10,
 21,
 20,
 14,
 13,
 10,
 18,
 15,
 11,
 12,
 13,
 24,
 20,
 11,
 12,
 10,
 20,
 14,
 17,
 16,
 13,
 14,
 12,
 14,
 10,
 12,
 19,
 17,
 11,
 9,
 7,
 27,
 14,
 12,
 9,
 9,
 13,
 11,
 8,
 6,
 7,
 20,
 20,
 19,
 14,
 16,
 13,
 12,
 12,
 10,
 9,
 29,
 22,
 19,
 16,
 9,
 14,
 12,
 8,
 10,
 10,
 12,
 13,
 12,
 8,
 8,
 14,
 15,
 9,
 11,
 9,
 18,
 16,
 10,
 9,
 8,
 19,
 16,
 14,
 11,
 11,
 11,
 11,
 10,
 8,
 10,
 15,
 11,
 11,
 6,
 8,
 17,
 16,
 11,
 12,
 9,
 17,
 16,
 12,
 10,
 11,
 40,
 12,
 12,
 10,
 8,
 22,
 23,
 14,
 13,
 11,
 17,
 14,
 13,
 10,
 9,
 22,
 19,
 18,
 7,
 8,
 21,
 23,
 16,
 6,
 6,
 35,
 21,
 17,
 8,
 6

In [14]:
maximum_length = max(lengths_of_captions)
captions_arrays = [0] * 256
list_of_captions_arrays = []

for array in captions:
    captions_arrays[:len(array)] = array
    new_caption_list = captions_arrays
    #print(new_caption_list)
    list_of_captions_arrays.append(list(new_caption_list))

#print(list_of_captions_arrays)


In [15]:
captions_tensor = torch.tensor(list_of_captions_arrays)

In [16]:
captions_tensor

tensor([[19621, 21030,   603,  ...,     0,     0,     0],
        [19621, 21030,  5957,  ...,     0,     0,     0],
        [19621, 21720,  5695,  ...,     0,     0,     0],
        ...,
        [19441, 18542,  5756,  ...,     0,     0,     0],
        [19441, 18542,  5251,  ...,     0,     0,     0],
        [19441, 18542, 18146,  ...,     0,     0,     0]])

In [17]:
class CaptionDataset(torch.utils.data.Dataset):

    def __init__(self, images, captions):
        self.images = images
        self.captions = captions

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        caption = self.captions[idx]
        return image, caption

In [35]:
dataset = CaptionDataset(df['image_name'], captions_tensor)

In [ ]:
for image, caption in dataset:
    image = image_data_preprocessor(image)
    plt.imshow(image[0])
    plt.show()
    #print(caption)

In [38]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle=False)

In [22]:
class CNNEncoder(nn.Module):
    def __init__(self, embed_s):
        super(CNNEncoder, self).__init__()
        """self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained = True, aux_logits = True)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_s)
        self.cnn = torch.nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.5),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5),
            #nn.Flatten()
        )"""
        self.cnn = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU()
            #nn.MaxPool2d(kernel_size=2),
        self.drp = nn.Dropout(0.5)
            
       
        #self.dropout = nn.Dropout(0.5)
        self.attention = nn.Linear(embed_s, embed_s)
        #self.softmax = nn.Softmax(dim=1)
        #self.relu = nn.ReLU()

    def forward(self, input_img):
        #print(input_img.shape)
        output_features = self.drp(self.relu(self.cnn(input_img)))
        #print(output_features.shape)
        #output_features = self.cnn(input_img)
        attention = self.attention(output_features)
        #feature = self.inception(input_img.unsqueeze(0))
        #attention_output = self.softmax(attention)
        #feature_embedding = output_features * attention_output
        """for name, param in self.inception.named_parameters():
            if "fc_weight" in name or "fc_bias" in name:
                param.requires_grad = True
            else:
                param.requires_grad = self.train_CNN"""

        return attention

    

In [23]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_s, hidden_s, vocab_s, num_layers):
        super(DecoderRNN, self).__init__()
        self.hidden_s = hidden_s
        self.embed = nn.Embedding(vocab_s, embed_s)
        self.lstm = nn.LSTM(embed_s, hidden_s, num_layers)
        self.linear = nn.Linear(hidden_s, vocab_s)
        self.dropout = nn.Dropout(0.5)
        #self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, features, captions):
        #h_0 = torch.zeros()
        embeddings = self.embed(captions)
        #embeddings = embeddings.detach().requires_grad_(True)
        #print( features.unsqueeze(0).shape, embeddings.shape)
        concatenated_embeddings = torch.cat((features, embeddings),dim = 0)
        #print(concatenated_embeddings.shape)
        hidden, _ = self.lstm(concatenated_embeddings)
        #print(hidden.shape)
        #hidden = hidden.view(-1, self.hidden_s)
        #print(hidden.shape)
        #hidden.requires_grad = True
        output = self.linear(hidden)
        #print(output.shape)
        #output = self.softmax(output)
        return output

In [24]:
class Encoder_Decoder(nn.Module):
    def __init__(self, embed_s, hidden_s, vocab_s, num_layers):
        super(Encoder_Decoder, self).__init__()
        self.encoder = CNNEncoder(embed_s)
        self.decoder = DecoderRNN(embed_s, hidden_s, vocab_s, num_layers)
    
    def forward(self, images, captions):
        #print(image.unsqueeze(0).shape)
        features = self.encoder(images)
        #print(features.shape, captions.shape)
        output = self.decoder(features, captions)
        #print(output.shape)
        return output

    """ def caption_image(self, image, vocabulary, max_l = 256):
        result_cap =[]
        with torch.no_grad():
            x = self.encoder(image).squeeze(0)
            states = None

            for _ in range(maximum_l):
                hiddens, states = self.decoder.lstm(x, states)
                output = self.decoder.linear(hiddens.unsqueeze(0))
                predicted = output.argmax(1)

                result_cap.append(predicted.item())
                x = self.decoder.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_cap]"""

In [43]:
embed_size = 256
hidden_size = 256
vocab_size = len(vocab)
num_layers = 1
learning_rate = 0.01
num_epochs = 500
encoder = CNNEncoder(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)
model = Encoder_Decoder(embed_size, hidden_size, vocab_size, num_layers)
loss_function = nn.CrossEntropyLoss()
#loss_function = loss_function.requires_grad_(True)
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)#, alpha= 0.99, eps = 1e-08)
#torch.set_grad_enabled(True)

In [44]:
#model.eval()
for epoch in range(1, num_epochs + 1):
    
    for i, (image, caption) in enumerate(train_dataloader):
        #print(image)
        optimizer.zero_grad()
        #image = image_data_preprocessor(image)
        image = image_data_preprocessor(str(image[0]))
        #image = image.to(device)
        #caption = caption.to(device)
        print(caption[:-1].shape)
        output = model(image, caption[:-1])
        
        #output = torch.tensor(output, requires_grad=True)
        #print(output.shape, caption.shape)
        #max_output = output.argmax(dim=2)
        output = output.permute(0, 2, 1)
        #print(output.shape, caption.shape)
        loss = loss_function(output, caption)
        
        loss.backward()
        optimizer.step()
        loss_train = loss.item()
        #if epoch == 1 or epoch % 10 == 0:
        print('{} Epoch {}, Training loss {}'.format(
        datetime.datetime.now(), epoch,
        loss_train/len(train_dataloader)))

torch.Size([9, 256])
2023-01-14 21:29:21.528162 Epoch 1, Training loss 0.000639862837792643
torch.Size([9, 256])
2023-01-14 21:29:23.750221 Epoch 1, Training loss 0.0006397465989837532
torch.Size([9, 256])
2023-01-14 21:29:26.004192 Epoch 1, Training loss 0.0006397247554491966
torch.Size([9, 256])


KeyboardInterrupt: 

Encoder_Decoder(
  (encoder): CNNEncoder(
    (cnn): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (drp): Dropout(p=0.5, inplace=False)
    (attention): Linear(in_features=256, out_features=256, bias=True)
  )
  (decoder): DecoderRNN(
    (embed): Embedding(220, 256)
    (lstm): LSTM(256, 256)
    (linear): Linear(in_features=256, out_features=220, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (softmax): Softmax(dim=1)
  )
)

In [ ]:
predicted_captions = []

for i, (image, caption) in enumerate(train_dataloader): 
        image = image_data_preprocessor(str(image[0]))
        predi = model(image, caption)
        preds = nn.Softmax(dim=1)
        pred = preds(predi[0])
        each_ind = pred.max(dim=1)
        each_seq = each_ind[1]
        predicted_captions.append(each_ind[1])

        

In [ ]:
#predicted_captions

In [ ]:
first_new_array = []
second_new_array = []
for i, image_tensor in enumerate(predicted_captions):
        first_new_array[:23] = predicted_captions[i][:23]
        second_new_array.append(first_new_array)
        

In [ ]:
second_new_array[0]

[tensor(141),
 tensor(0),
 tensor(0),
 tensor(137),
 tensor(58),
 tensor(0),
 tensor(58),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(58),
 tensor(141),
 tensor(67),
 tensor(0),
 tensor(141),
 tensor(141),
 tensor(23),
 tensor(137),
 tensor(137),
 tensor(141),
 tensor(58),
 tensor(52),
 tensor(58)]

In [ ]:

second_sentence = []
for i in range(len(second_new_array)):
    first_sentence = []
    for ind in second_new_array[i]:
        word = vocab[ind]
        first_sentence.append(word)
    second_sentence.append(first_sentence)
        #print(sentence)

In [ ]:
print(second_sentence[0])
for cap in df['comment']:
    print(cap)


['stands', 'friends', 'stands', 'bushes', 'to', 'walking', 'look', 'cleaning', 'gate', 'Three', 'gate', 'outside', 'outside', 'Several', 'hip', 'talking', 'street', 'outside', 'out', 'suit', 'dressed', 'outside', 'past']
 Two young guys with shaggy hair look at their hands while hanging out in the yard .
 Two young , White males are outside near many bushes .
 Two men in green shirts are standing in a yard .
 A man in a blue shirt standing in a garden .
 Two friends enjoy time spent together .
 Several men in hard hats are operating a giant pulley system .
 Workers look down from up above on a piece of equipment .
 Two men working on a machine wearing hard hats .
 Four men on top of a tall structure .
 Three men on a large rig .
 A child in a pink dress is climbing up a set of stairs in an entry way .
 A little girl in a pink dress going into a wooden cabin .
 A little girl climbing the stairs to her playhouse .
 A little girl climbing into a wooden playhouse 
 A girl going into a wood

In [ ]:
#predicted_captions[0][0]
new_array = []
new_array[:23] = predicted_captions[0][:23]
len(new_array)

23

In [ ]:
sentence = []
for i in new_array:
    word = vocab[i]
    sentence.append(word)